# OpenAlex RAG

Author: Alex Davis

Date: 09/03/2025

The purpose of this script is to load a vector database, create a RAG capability, test, and compare outputs to a normal generative response.

## Load Packages 

In [0]:
%pip install faiss-cpu
%pip install -qU langchain-community faiss-cpu
%pip install --upgrade --quiet  langchain langchain-huggingface sentence_transformers
%pip install -qU langchain-openai

In [0]:
%restart_python

In [0]:
#import packages
import os
import faiss
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

## Load Models and Embeddings

In [0]:
#load embedding model
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

In [0]:
#load vector DB
db = FAISS.load_local(
    "Data/faiss_index", embeddings, allow_dangerous_deserialization=True)

In [0]:
#set API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY","API KEY") 

#load llm
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

## Test Base LLM

In [0]:
#test llm response
llm.invoke("Computer vision")

' technology has been around for a while, but it has recently gained more traction due to advancements in machine learning algorithms and the availability of large amounts of data. This technology allows computers to analyze, understand, and interpret visual data, such as images and videos.\n\nThere are several key areas where computer vision technology is being applied:\n\n1. Object Detection and Recognition\nOne of the main applications of computer vision is object detection and recognition. This involves the ability of computers to identify and classify objects in images or videos. This technology is used in self-driving cars, facial recognition systems, and security surveillance systems.\n\n2. Image and Video Understanding\nComputer vision technology can also analyze and understand the content of images and videos. This allows computers to extract information from visual data, such as identifying patterns or objects, and recognizing emotions and actions.\n\n3. Medical Imaging\nIn the medical field, computer vision is used for tasks such as medical image analysis and diagnosis. This technology can help doctors detect and diagnose diseases, track the progression of treatments, and assist in surgical procedures.\n\n4. Augmented and Virtual Reality\nComputer vision is also a key component in augmented and virtual reality technologies. It enables these systems to interact with the real world and create immersive experiences by recognizing and tracking objects and movements.\n\n5.'

In [0]:
#test llm response
llm.invoke("What are the most recent advancements in computer vision?")

'\n\n1. Deep Learning: Deep learning, a subset of machine learning, has shown significant progress in computer vision tasks such as object detection, recognition, and image classification. It uses neural networks with multiple hidden layers to learn and extract features from images, leading to more accurate and efficient results.\n\n2. Generative Adversarial Networks (GANs): GANs are a type of deep learning algorithm that generates new images by learning from a large dataset. They have been used in tasks such as image synthesis, super-resolution, and image-to-image translation, and have shown impressive results in creating realistic images.\n\n3. Convolutional Neural Networks (CNNs): CNNs are a type of deep learning algorithm that has revolutionized the field of computer vision. They are highly effective in extracting features from images and have been used in various tasks such as image classification, object detection, and segmentation.\n\n4. Transfer Learning: Transfer learning allows a pre-trained model to be used on a different task or dataset without starting from scratch. It has shown promising results in computer vision tasks, especially for tasks with limited training data.\n\n5. Image Segmentation: With advancements in deep learning, image segmentation has become more accurate and efficient. It involves dividing an image into different regions or segments to identify objects'

## Create Retriever and RAG Pipeline

In [0]:
#test that vector database is working
retriever = db.as_retriever(search_kwargs={"k": 3})

In [0]:
#create a prompt template
template = """<|user|>
Relevant information:
{context}

Provide a concise answer to the following question using relevant information provided above:
{question}
If the information above does not answer the question, say that you do not know. Keep answers to 3 sentences or shorter.<|end|>
<|assistant|>"""

#define prompt template
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"])

In [0]:
#create RAG pipeline
rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True,
                                  chain_type_kwargs={"prompt": prompt}, verbose = True)

## Test RAG Pipeline

In [0]:
#test RAG
rag.invoke("computer vision")



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'computer vision',
 'result': '\nComputer vision is a field of artificial intelligence that enables computer software to understand and learn from visual data. It has applications in various fields such as self-driving cars, facial recognition, and healthcare. Deep learning techniques, specifically convolutional neural networks, have greatly advanced the capabilities of computer vision. Current research is focused on implementing and optimizing these algorithms on hardware accelerators for real-time and energy-efficient operations.',
 'source_documents': [Document(id='783', metadata={'title': 'FACE DETECTION AND RECOGNITION USING OPENCV', 'year': 2020}, page_content='Computer Vision is one of the most fascinating and challenging tasks in the field of Artificial Intelligence.Computer Vision serves as a link between computer software and the visuals we see around us.It enables computer software to comprehend and learn about the visuals in its environment.As an example: The fruit is determined by its color, shape, and size.This job may seem simple for the human brain, but in the Computer Vision pipeline, we first collect data, then conduct data processing operations, and then train and educate the model to learn how to differentiate between fruits based on size, shape, and color.The main goal is to identify and comprehend the images and offer new images that are more useful for us in different life fieldsThe term "OpenCV" is an abbreviation for "open source computer vision."The architecture is made up of software, databases, and plugins that are pre-programmed with support for integrating computer vision applications [3].It is one of the most used toolkits with a large developer group.It is well-known for the size at which it builds realworld usage cases for industrial use.OpenCV follows C/C++, Python, Java programming languages and can be used to build computer vision software for desktop and smartphone platforms such as Windows, Linux, macOS, Android, and iOS.The most recent releases are OpenCV-4.5.2 and OpenCV-3.4.14.It is free and open-source, as well as simple to use and install.It is intended for numerical productivity with a heavy emphasis on real-time applications.The first version was in the C programming language; however, its success increased with the release of Version 2.0, which had a C++ implementation [2].C++ is used to create new features.OpenCV can be downloaded for free from http://opencv.org.This platform includes the most recent distribution update (version: 4.5.2) as well as older iterations.Photos must be in BGR or Grayscale format in order to be displayed or saved via OpenCV.Otherwise, unfavorable outcomes could occur [1].Face detection is a form of computer vision that aids in detecting and visualizing facial features in captured pictures or real-time videos.This type of object detection technique detects instances of semantic artifacts of a given class (such as people, cars, and houses) in digital pictures and videos.Face recognition has become increasingly important as technology has advanced, especially in fields such as photography, defense, and marketing [4], [5].'),
  Document(id='220', metadata={'title': 'Image Classification Based On CNN: A Survey', 'year': 2021}, page_content="Computer vision is one of the fields of computer science that is one of the most powerful and persuasive types of artificial intelligence. It is similar to the human vision system, as it enables computers to recognize and process objects in pictures and videos in the same way as humans do. Computer vision technology has rapidly evolved in many fields and contributed to solving many problems, as computer vision contributed to self-driving cars, and cars were able to understand their surroundings. The cameras record video from different angles around the car, then a computer vision system gets images from the video, and then processes the images in real-time to find roadside ends, detect other cars, and read traffic lights, pedestrians, and objects. Computer vision also contributed to facial recognition; this technology enables computers to match images of people’s faces to their identities. which these algorithms detect facial features in images and then compare them with databases. Computer vision also play important role in Healthcare, in which algorithms can help automate tasks such as detecting Breast cancer, finding symptoms in x-ray, cancerous moles in skin images, and MRI scans. Computer vision also contributed to many fields such as image classification, object discovery, motion recognition, subject tracking, and medicine. The rapid development of artificial intelligence is making machine learning more important in his field of research. Use algorithms to find out every bit of data and predict the outcome. This has become an important key to unlocking the door to AI. If we had looked to deep learning concept, we find deep learning is a subset of machine learning, algorithms inspired by structure and function of the human brain called artificial neural networks, learn from large amounts of data. Deep learning algorithm perform a task repeatedly, each time tweak it a little to improve the outcome. So, the development of computer vision was due to deep learning. Now we'll take a tour around the convolution neural networks, let us say that convolutional neural networks are one of the most powerful supervised deep learning models (abbreviated as CNN or ConvNet). This name ;convolutional ; is a token from a mathematical linear operation between matrixes called convolution. CNN structure can be used in a variety of real-world problems including, computer vision, image recognition, natural language processing (NLP), anomaly detection, video analysis, drug discovery, recommender systems, health risk assessment, and time-series forecasting. If we look at convolutional neural networks, we see that CNN are similar to normal neural networks, the only difference between CNN and ANN is that CNNs are used in the field of pattern recognition within images mainly. This allows us to encode the features of an image into the structure, making the network more suitable for image-focused tasks, with reducing the parameters required to set-up the model. One of the advantages of CNN that it has an excellent performance in machine learning problems. So, we will use CNN as a classifier for image classification. So, the objective of this paper is that we will talk in detail about image classification in the following sections."),
  Document(id='5322', metadata={'title': 'Computer vision algorithms and hardware implementations: A survey', 'year': 2019}, page_content='The field of computer vision is experiencing a great-leap-forward development today. This paper aims at providing a comprehensive survey of the recent progress on computer vision algorithms and their corresponding hardware implementations. In particular, the prominent achievements in computer vision tasks such as image classification, object detection and image segmentation brought by deep learning techniques are highlighted. On the other hand, review of techniques for implementing and optimizing deep-learning-based computer vision algorithms on GPU, FPGA and other new generations of hardware accelerators are presented to facilitate real-time and/or energy-efficient operations. Finally, several promising directions for future research are presented to motivate further development in the field.')]}

In [0]:
#test rag response
rag.invoke("What are the most recent advancements in computer vision?")



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What are the most recent advancements in computer visions?',
 'result': " The most recent advancements in computer vision include the emergence of large language models equipped with vision capabilities, such as OpenAI's GPT-4V, Google's Bard AI, and Microsoft's Bing AI. These models are able to analyze images and have the ability to access real-time information, making them directly embedded in many applications. Further advancements are expected as AI continues to rapidly evolve.",
 'source_documents': [Document(id='980', metadata={'title': 'Image and data mining in reticular chemistry powered by GPT-4V', 'year': 2024}, page_content='The integration of artificial intelligence into scientific research opens new avenues with the advent of GPT-4V, a large language model equipped with vision capabilities.'),
  Document(id='3691', metadata={'title': 'Viewpoint: When Will AI Exceed Human Performance? Evidence from AI Experts', 'year': 2018}, page_content='&#x0D; &#x0D; &#x0D; &#x0D; Advances in artificial intelligence (AI) will transform modern life by reshaping transportation, health, science, finance, and the military. To adapt public policy, we need to better anticipate these advances. Here we report the results from a large survey of machine learning researchers on their beliefs about progress in AI. Researchers predict AI will outperform humans in many activities in the next ten years, such as translating languages (by 2024), writing high-school essays (by 2026), driving a truck (by 2027), working in retail (by 2031), writing a bestselling book (by 2049), and working as a surgeon (by 2053). Researchers believe there is a 50% chance of AI outperforming humans in all tasks in 45 years and of automating all human jobs in 120 years, with Asian respondents expecting these dates much sooner than North Americans. These results will inform discussion amongst researchers and policymakers about anticipating and managing trends in AI.&#x0D; &#x0D; &#x0D; This article is part of the special track on AI and Society.&#x0D; &#x0D; &#x0D; &#x0D; &#x0D; &#x0D;'),
  Document(id='1580', metadata={'title': 'Teaching and assessment of the future today: higher education and AI', 'year': 2023}, page_content='Artificial intelligence (AI), once a subject of science fiction, is now a tangible, disruptive force in teaching and learning. In an educational setting, generative large language models (LLM), such as OpenAI’s ChatGPT, perform and supplement tasks that usually require human thought, such as data analysis, understanding complex ideas, problem-solving, coding and producing written outputs. AI advances are moving quickly. From the emergence of ChatGPT 3.5 in November 2022, we have witnessed the arrival of other progressive language models, like OpenAI’s GPT-4, Google’s Bard AI and Microsoft’s Bing AI. Most recently, AIs gained the ability to access real-time information, analyse images and are becoming directly embedded in many applications.')]}